# Composites

In [66]:
import glob, os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot

from pathlib import Path
from IPython.display import display

## Set directory

In [67]:
computer_path = '/export/miro/ndeffense/LBRAT2104/'
grp_letter    = 'X'

data_path = f'{computer_path}data/'                     # Directory with data shared by the assistant
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'  # Directory for all work files

# Input directories
ndvi_path    = f'{work_path}NDVI/'

# Output directory
composite_path = f'{work_path}COMPOSITE/'

Path(composite_path).mkdir(parents=True, exist_ok=True)

## Set parameters

Method :
- mean
- median
- maximum
- minimum

In [68]:
method = 'minimum'

## Set filenames

In [69]:
composite_tif = f'{composite_path}composite_{method}.tif'

In [70]:
list_src_arr = []

list_im = sorted(glob.glob(f'{ndvi_path}*.tif'))

for im_file in list_im:

    src = rasterio.open(im_file, "r")
    im = src.read(1)
    list_src_arr.append(im)
    src.close()
    
print(f'Shape of features : {im.shape}')
print(f'Number of features : {len(list_src_arr)}')

Shape of features : (570, 986)
Number of features : 12


In [71]:
im_arr = np.dstack(list_src_arr)

print(im_arr.shape)
print(f'There are {im_arr.shape[2]} features')
print(f'The features type is : {im_arr.dtype}')

(570, 986, 12)
There are 12 features
The features type is : float64


In [72]:
if method == 'median':
    compo_arr = np.nanmedian(im_arr, axis=2)

elif method == 'mean':
    compo_arr = np.nanmean(im_arr, axis=2)

elif method == 'std':
    compo_arr = np.nanstd(im_arr, axis=2)

elif method == 'minimum':
    compo_arr = np.nanmin(im_arr, axis=2)

elif method == 'maximum':
    compo_arr = np.nanmax(im_arr, axis=2)

else:
    print(f'Method not available !')

print(compo_arr)

[[0.42912925 0.43237788 0.37547408 ... 0.5170441  0.49381541 0.34527089]
 [0.43097239 0.43376623 0.42655262 ... 0.49025202 0.32620818 0.20146688]
 [0.43283582 0.44285215 0.44069015 ... 0.33983287 0.18755803 0.14951095]
 ...
 [0.58860104 0.55236576 0.51199638 ... 0.11897106 0.13888889 0.13902205]
 [0.60609911 0.56308411 0.54875177 ... 0.12505643 0.14905399 0.14137116]
 [0.60179833 0.596366   0.58564103 ... 0.11681173 0.13850416 0.14312354]]


/export/miro/ndeffense/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: All-NaN slice encountered
  # This is added back by InteractiveShellApp.init_path()


In [73]:
img_temp_tif = list_im[0]

with rasterio.open(img_temp_tif) as src:
    profile = src.profile

print(profile)

with rasterio.open(composite_tif, "w", **profile) as dst:
    dst.write(compo_arr, 1)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': -10000.0, 'width': 986, 'height': 570, 'count': 1, 'crs': CRS({'init': 'epsg:32631'}), 'transform': Affine(10.0, 0.0, 627260.0,
       0.0, -10.0, 5596180.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
